In [ ]:
import zipfile
import json
import pandas as pd
import numpy as np
from PIL import Image
import zipfile
import os
import shutil


In [25]:
pd.set_option("display.max_columns", None)  # 열 전체 표시
pd.set_option("display.width", None)  # 가로 폭 제한 제거

### 0. data load & 일부 스타일 

In [26]:
zip_path = (
    "C:/Users/Admin/Desktop/PROJ/data/01_raw/K-Fashion 이미지/Training/라벨링데이터.zip"
)
styles = [
    "로맨틱/",
    "섹시/",
    "소피스트케이티드/",
    "스포티/",
    "클래식/",
    "젠더리스/",
    "아방가르드/",
]

json_list = []

with zipfile.ZipFile(zip_path, "r") as z:
    for info in z.infolist():
        real_name = info.filename.encode("cp437").decode("cp949")

        # 위 스타일 폴더들 중 하나에 속하면서 .json 인 파일만
        if any(s in real_name for s in styles) and real_name.endswith(".json"):
            with z.open(info.filename) as f:
                data = json.load(f)
                json_list.append(data)

print(f"불러온 json 개수: {len(json_list)}")

불러온 json 개수: 182290


In [27]:
# DataFrame 생성
df = pd.json_normalize(json_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182290 entries, 0 to 182289
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   이미지 정보.이미지 식별자               182290 non-null  int64 
 1   이미지 정보.이미지 높이                182290 non-null  int64 
 2   이미지 정보.이미지 파일명               182290 non-null  object
 3   이미지 정보.이미지 너비                182290 non-null  int64 
 4   데이터셋 정보.파일 생성일자              182290 non-null  object
 5   데이터셋 정보.데이터셋 상세설명.렉트좌표.아우터   182290 non-null  object
 6   데이터셋 정보.데이터셋 상세설명.렉트좌표.하의    182290 non-null  object
 7   데이터셋 정보.데이터셋 상세설명.렉트좌표.원피스   182290 non-null  object
 8   데이터셋 정보.데이터셋 상세설명.렉트좌표.상의    182290 non-null  object
 9   데이터셋 정보.데이터셋 상세설명.폴리곤좌표.아우터  182290 non-null  object
 10  데이터셋 정보.데이터셋 상세설명.폴리곤좌표.하의   182290 non-null  object
 11  데이터셋 정보.데이터셋 상세설명.폴리곤좌표.원피스  182290 non-null  object
 12  데이터셋 정보.데이터셋 상세설명.폴리곤좌표.상의   182290 non-null  object
 13  데이터셋 정보.데이터셋 상

### 1. 전처리

##### - 필요한 컬럼

In [28]:
# 필요한 컬럼만
df_filter = df.iloc[:, [0, 1, 2, 3, 6, 7, 8, 13, 14, 15, 16, 17]]

##### - 빈 딕셔너리 nan으로

In [29]:
# 빈 딕셔너리 nan으로
df_filter = df_filter.copy()
obj_cols = df_filter.select_dtypes(include="object").columns


def is_empty_list_dict(x):
    return isinstance(x, list) and (x == [] or x == [{}])


mask = df_filter[obj_cols].applymap(is_empty_list_dict)

df_filter.loc[:, obj_cols] = df_filter[obj_cols].mask(mask, np.nan)

C:\Users\Admin\AppData\Local\Temp\ipykernel_25100\393704689.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = df_filter[obj_cols].applymap(is_empty_list_dict)


##### - 아우터 있는 데이터 제거

In [30]:
# '아우터' 값이 존재하는 행 제거
df_filter = df_filter[df_filter["데이터셋 정보.데이터셋 상세설명.라벨링.아우터"].isna()]
print("아우터 있는 데이터 제거 후 개수:", len(df_filter))

아우터 있는 데이터 제거 후 개수: 140058


##### - [상의&하의 or 원피스]의 데이터만


In [31]:
# 상의 & 하의 모두 또는 원피스만 존재하는 행
df_filter = df_filter[
    (
        df_filter["데이터셋 정보.데이터셋 상세설명.라벨링.상의"].notna()
        & df_filter["데이터셋 정보.데이터셋 상세설명.라벨링.하의"].notna()
        & df_filter["데이터셋 정보.데이터셋 상세설명.라벨링.원피스"].isna()
    )
    | (
        df_filter["데이터셋 정보.데이터셋 상세설명.라벨링.원피스"].notna()
        & df_filter["데이터셋 정보.데이터셋 상세설명.라벨링.상의"].isna()
        & df_filter["데이터셋 정보.데이터셋 상세설명.라벨링.하의"].isna()
    )
]

##### - 라벨링 컬럼 파싱

In [32]:
# 컬럼명 변경
df_filter.columns = [
    "식별자",
    "이미지높이",
    "파일명",
    "이미지너비",
    "렉트좌표_하의",
    "렉트좌표_원피스",
    "렉트좌표_상의",
    "라벨링_스타일",
    "라벨링_아우터",
    "라벨링_하의",
    "라벨링_원피스",
    "라벨링_상의",
]
df_filter = df_filter.drop(["라벨링_아우터"], axis=1)

In [33]:
# 파싱 함수 생성
def expand_dict(val):
    if isinstance(val, list) and len(val) > 0 and isinstance(val[0], dict):
        return val[0]
    return {}

In [34]:
# 스타일, 서브스타일 파싱
parsed = df_filter["라벨링_스타일"].apply(expand_dict).apply(pd.Series)
df_parsed = pd.concat([df_filter, parsed], axis=1)

# 하의 라벨링 파싱
parsed_add = df_filter["라벨링_하의"].apply(expand_dict).apply(pd.Series)

parsed_add["소재"] = parsed_add["소재"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)
parsed_add["프린트"] = parsed_add["프린트"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)
parsed_add = parsed_add.add_prefix("하의_")
df_parsed = pd.concat([df_parsed, parsed_add], axis=1)


In [35]:
# 원피스 라벨링 파싱
parsed_add = df_filter["라벨링_원피스"].apply(expand_dict).apply(pd.Series)

parsed_add["디테일"] = parsed_add["디테일"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)
parsed_add["소재"] = parsed_add["소재"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)
parsed_add["프린트"] = parsed_add["프린트"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)

parsed_add = parsed_add.add_prefix("원피스_")
df_parsed = pd.concat([df_parsed, parsed_add], axis=1)

In [36]:
# 상의 라벨링 파싱
parsed_add = df_filter["라벨링_상의"].apply(expand_dict).apply(pd.Series)

parsed_add["소재"] = parsed_add["소재"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)
parsed_add["프린트"] = parsed_add["프린트"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)

parsed_add = parsed_add.add_prefix("상의_")

df_parsed = pd.concat([df_parsed, parsed_add], axis=1)

In [37]:
# 라벨링_로 시작하는 컬럼명 제거 (파싱전의 컬럼 제거)
label_cols = [col for col in df_parsed.columns if col.startswith("라벨링_")]
df_parsed = df_parsed.drop(columns=label_cols)

In [38]:
df_parsed.shape

(98690, 39)

##### _옷깃, _서브색상 컬럼 제거

In [39]:
suffixes = ('_옷깃', '_서브색상')
cols_to_drop = [col for col in df_parsed.columns if col.endswith(suffixes)]
df_parsed = df_parsed.drop(columns=cols_to_drop)

##### - 중복 옷 제거

In [40]:
# 1차 : 연속된 행의 정보가 80% 이상 유사할시 제거
# ---------------------------------------
# 1. 비교 대상 컬럼 자동 선택
# ---------------------------------------
target_cols = [
    col for col in df_parsed.columns
    if col.startswith(("상의_", "하의_", "원피스_"))
]

print("비교 대상 컬럼 개수:", len(target_cols))

# ---------------------------------------
# 2. 결측치 통일
# ---------------------------------------
df_compare = df_parsed[target_cols].fillna("##NULL##")

# ---------------------------------------
# 3. 이전 행과의 유사도 계산
# ---------------------------------------
df_shift = df_compare.shift(1)

# True/False 비교 결과
match_matrix = (df_compare == df_shift)

# 행별 유사도 계산
similarity_ratio = match_matrix.sum(axis=1) / len(target_cols)

# ---------------------------------------
# 4. 유사도 70% 이상 제거
# ---------------------------------------
threshold = 0.7

df_filtered = df_parsed[similarity_ratio < threshold].reset_index(drop=True)

print("기존 행 수:", len(df_parsed))
print("필터링 후 행 수:", len(df_filtered))


비교 대상 컬럼 개수: 25
기존 행 수: 98690
필터링 후 행 수: 8248


In [41]:
# 2차 : 연속된 행의 제품명과 스타일이 동일한 경우 제거
df_filtered["제품명_앞"] = (
    df_filtered["파일명"]
    .str.split("_")
    .str[0]
    .str.strip()
    .str.lower()
)

df_filtered["스타일_clean"] = (
    df_filtered["스타일"]
    .str.strip()
    .str.lower()
)

mask = ~(
    (df_filtered["제품명_앞"] == df_filtered["제품명_앞"].shift(1)) &
    (df_filtered["스타일_clean"] == df_filtered["스타일_clean"].shift(1))
)

df_filtered_2 = df_filtered[mask].reset_index(drop=True)
df_filtered_2 = df_filtered_2.drop(columns=["제품명_앞", "스타일_clean"])

print("기존 행 수:", len(df_filtered))
print("필터링 후 행 수:", len(df_filtered_2))

기존 행 수: 8248
필터링 후 행 수: 6109


### 2. [UI용]

##### - 이미지 자르기(crop)

In [42]:
# 좌표정보만
df_xy = df_filtered_2[
    ["식별자", "스타일", "렉트좌표_상의", "렉트좌표_하의", "렉트좌표_원피스"]
]
# 식별자 컬럼을 파일명으로
df_xy["식별자"] = df_xy["식별자"].astype(str) + ".jpg"

C:\Users\Admin\AppData\Local\Temp\ipykernel_25100\2606698141.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xy["식별자"] = df_xy["식별자"].astype(str) + ".jpg"


In [43]:
# 자른 이미지 저장할 폴더
base_save_folder = "C:/Users/Admin/Desktop/PROJ/data/02_cleaned/clothes/1.cropped_images/"
for part in ["상의", "하의", "원피스"]:
    os.makedirs(os.path.join(base_save_folder, part), exist_ok=True)

In [44]:
# 각 스타일이 어느 zip에 있는지
style_to_zip = {
    "로맨틱": "C:/Users/Admin/Desktop/PROJ/data/01_raw/K-Fashion 이미지/Training/원천데이터_1.zip",
    "섹시": "C:/Users/Admin/Desktop/PROJ/data/01_raw/K-Fashion 이미지/Training/원천데이터_1.zip",
    "소피스트케이티드": "C:/Users/Admin/Desktop/PROJ/data/01_raw/K-Fashion 이미지/Training/원천데이터_1.zip",
    "스포티": "C:/Users/Admin/Desktop/PROJ/data/01_raw/K-Fashion 이미지/Training/원천데이터_3.zip",
    "아방가르드": "C:/Users/Admin/Desktop/PROJ/data/01_raw/K-Fashion 이미지/Training/원천데이터_3.zip",
    "클래식": "C:/Users/Admin/Desktop/PROJ/data/01_raw/K-Fashion 이미지/Training/원천데이터_3.zip",
    "젠더리스": "C:/Users/Admin/Desktop/PROJ/data/01_raw/K-Fashion 이미지/Training/원천데이터_3.zip",
}

In [45]:
# crop function
def crop_and_save_by_part(df, zip_folder_map, base_save_folder, batch_size=100):
    zip_cache = {}

    for start in range(0, len(df), batch_size):
        batch = df.iloc[start : start + batch_size]
        print(f"Processing batch {start} ~ {start + len(batch) - 1}")

        for idx, row in batch.iterrows():
            style = row["스타일"]
            zip_path = zip_folder_map.get(style)
            if zip_path is None:
                print(f"{style} 카테고리의 zip 파일이 정의되어 있지 않습니다.")
                continue

            # zip 캐시
            if zip_path not in zip_cache:
                zip_cache[zip_path] = zipfile.ZipFile(zip_path, "r")
            z = zip_cache[zip_path]

            img_files = [name for name in z.namelist() if name.endswith(row["식별자"])]
            if img_files:
                img_zip_path = img_files[0]
            if img_zip_path in z.namelist():
                with z.open(img_zip_path) as f:
                    img = Image.open(f).convert("RGB")
                    img_w, img_h = img.size

                    # 컬럼별 처리
                    for col in ["렉트좌표_상의", "렉트좌표_하의", "렉트좌표_원피스"]:
                        rects = row.get(col)
                        if pd.isna(rects):
                            continue
                        part_name = col.split("_")[
                            -1
                        ]  # 컬럼명에서 상의, 하의, 원피스 추출
                        save_folder = os.path.join(base_save_folder, part_name)

                        for i, r in enumerate(rects):

                            # 키 확인
                            if not all(
                                k in r for k in ["X좌표", "Y좌표", "가로", "세로"]
                            ):
                                continue

                            # 좌표 정수화 + 보정
                            x = max(0, int(float(r["X좌표"])))
                            y = max(0, int(float(r["Y좌표"])))
                            w = int(float(r["가로"]))
                            h = int(float(r["세로"]))

                            if w <= 0 or h <= 0:
                                continue

                            x2 = min(img_w, x + w)
                            y2 = min(img_h, y + h)

                            if x >= x2 or y >= y2:
                                continue
                            cropped = img.crop((x, y, x2, y2))
                            save_path = os.path.join(
                                save_folder,
                                f"{style}_{row['식별자'].replace('.jpg','')}_{part_name}.jpg",
                            )
                            cropped.save(save_path)
            else:
                print(f"{img_zip_path} 파일이 zip 안에 존재하지 않습니다.")

    # zip 닫기
    for z in zip_cache.values():
        z.close()

In [46]:
# 이미지 crop 실행
crop_and_save_by_part(df_xy, style_to_zip, base_save_folder, batch_size=100)

Processing batch 0 ~ 99
Processing batch 100 ~ 199
Processing batch 200 ~ 299
Processing batch 300 ~ 399
Processing batch 400 ~ 499
Processing batch 500 ~ 599
Processing batch 600 ~ 699
Processing batch 700 ~ 799
Processing batch 800 ~ 899
Processing batch 900 ~ 999
Processing batch 1000 ~ 1099
Processing batch 1100 ~ 1199
Processing batch 1200 ~ 1299
Processing batch 1300 ~ 1399
Processing batch 1400 ~ 1499
Processing batch 1500 ~ 1599
Processing batch 1600 ~ 1699
Processing batch 1700 ~ 1799
Processing batch 1800 ~ 1899
Processing batch 1900 ~ 1999
Processing batch 2000 ~ 2099
Processing batch 2100 ~ 2199
Processing batch 2200 ~ 2299
Processing batch 2300 ~ 2399
Processing batch 2400 ~ 2499
Processing batch 2500 ~ 2599
Processing batch 2600 ~ 2699
Processing batch 2700 ~ 2799
Processing batch 2800 ~ 2899
Processing batch 2900 ~ 2999
Processing batch 3000 ~ 3099
Processing batch 3100 ~ 3199
Processing batch 3200 ~ 3299
Processing batch 3300 ~ 3399
Processing batch 3400 ~ 3499
Processi

##### - 이미지 리사이즈

In [47]:
# ✅ 비교할 폴더 3개
folders = [
    r"C:/Users/Admin/Desktop/PROJ/data/02_cleaned/clothes/1.cropped_images/상의",
    r"C:/Users/Admin/Desktop/PROJ/data/02_cleaned/clothes/1.cropped_images/하의",
    r"C:/Users/Admin/Desktop/PROJ/data/02_cleaned/clothes/1.cropped_images/원피스"
]

In [48]:
# ✅ jpg 전용
image_extensions = ('.jpg', '.jpeg')

max_width = 0
max_height = 0

max_width_image = ""
max_height_image = ""

total_images = 0

for folder in folders:
    for root, dirs, files in os.walk(folder):
        for file in files:

            if file.lower().endswith(image_extensions):
                total_images += 1
                file_path = os.path.join(root, file)

                try:
                    with Image.open(file_path) as img:
                        w, h = img.width, img.height

                        # 최대 너비 업데이트
                        if w > max_width:
                            max_width = w
                            max_width_image = file_path

                        # 최대 높이 업데이트
                        if h > max_height:
                            max_height = h
                            max_height_image = file_path

                except Exception as e:
                    print(f"이미지 열기 실패: {file_path} → {e}")

print("\n✅ 전체 jpg 이미지 분석 결과")
print(f"총 이미지 개수: {total_images}")

print(f"\n📌 최대 Width: {max_width} px")
print(f"해당 이미지: {max_width_image}")

print(f"\n📌 최대 Height: {max_height} px")
print(f"해당 이미지: {max_height_image}")



✅ 전체 jpg 이미지 분석 결과
총 이미지 개수: 10140

📌 최대 Width: 797 px
해당 이미지: C:/Users/Admin/Desktop/PROJ/data/02_cleaned/clothes/1.cropped_images/원피스\로맨틱_118031_원피스.jpg

📌 최대 Height: 1262 px
해당 이미지: C:/Users/Admin/Desktop/PROJ/data/02_cleaned/clothes/1.cropped_images/하의\소피스트케이티드_523387_하의.jpg


In [49]:
# ✅ 결과 저장 폴더
output_base = r"C:/Users/Admin/Desktop/PROJ/data/02_cleaned/clothes/2.resized_images"
os.makedirs(output_base, exist_ok=True)

# ✅ 목표 크기
TARGET_WIDTH = max_width
TARGET_HEIGHT = max_height

image_extensions = ('.jpg', '.jpeg')

def resize_and_pad(image, target_w, target_h):
    original_w, original_h = image.size

    # ✅ 비율 유지
    ratio = min(target_w / original_w, target_h / original_h)
    new_size = (int(original_w * ratio), int(original_h * ratio))

    resized_img = image.resize(new_size, Image.LANCZOS)

    # ✅ 흰색 배경
    new_image = Image.new("RGB", (target_w, target_h), (255, 255, 255))

    # ✅ 중앙 정렬
    paste_x = (target_w - new_size[0]) // 2
    paste_y = (target_h - new_size[1]) // 2

    new_image.paste(resized_img, (paste_x, paste_y))

    return new_image


count = 0

for folder in folders:
    category = os.path.basename(folder)  # 상의 / 하의 / 원피스
    category_output = os.path.join(output_base, category)
    os.makedirs(category_output, exist_ok=True)

    for root, _, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(image_extensions):

                input_path = os.path.join(root, file)

                # ✅ 파일명 중복 방지를 위해 prefix 추가
                new_filename = f"{file}"
                output_path = os.path.join(category_output, new_filename)

                try:
                    with Image.open(input_path) as img:
                        final_img = resize_and_pad(img, TARGET_WIDTH, TARGET_HEIGHT)
                        final_img.save(output_path, quality=95)
                        count += 1

                except Exception as e:
                    print(f"실패: {input_path} → {e}")

print(f"\n✅ 총 {count}개의 이미지가 {TARGET_WIDTH}x{TARGET_HEIGHT} 크기로 통일되어 저장되었습니다.")



✅ 총 10140개의 이미지가 797x1262 크기로 통일되어 저장되었습니다.


##### - 카테고리/색상별 폴더화

In [50]:
#base_dir = r"C:/Users/Admin/Desktop/이정수/LG U+/프로젝트3/data" # 정수
base_dir = r"C:/Users/Admin/Desktop/PROJ/data/02_cleaned/clothes" # 원선

cropped_dir = os.path.join(base_dir, "2.resized_images")
output_dir = os.path.join(base_dir, "3.categorized_images")

os.makedirs(output_dir, exist_ok=True)

def safe_str(x):
    if pd.isna(x) or x in ["", None]:
        return "unknown"
    return str(x).strip().replace("/", "_")


for idx, row in df_filtered_2.iterrows():

    style = safe_str(row["스타일"])
    ident = safe_str(row["식별자"])

    # ===============================
    # 1) 원피스 처리 (copy only)
    # ===============================
    if not pd.isna(row["원피스_카테고리"]):

        filename = f"{style}_{ident}_원피스.jpg"
        src_path = os.path.join(cropped_dir, "원피스", filename)

        if os.path.exists(src_path):
            category = safe_str(row["원피스_카테고리"])
            color = safe_str(row["원피스_색상"])

            dst_folder = os.path.join(output_dir, "원피스", category, color)
            os.makedirs(dst_folder, exist_ok=True)

            shutil.copy(src_path, os.path.join(dst_folder, filename))

        continue  # 원피스는 여기서 끝

    # ===============================
    # 2) 투피스(상의 + 하의) 처리
    # ===============================

    # ▶ 상의 copy
    if not pd.isna(row["상의_카테고리"]):
        filename = f"{style}_{ident}_상의.jpg"
        src_path = os.path.join(cropped_dir, "상의", filename)

        if os.path.exists(src_path):
            category = safe_str(row["상의_카테고리"])
            color = safe_str(row["상의_색상"])

            dst_folder = os.path.join(output_dir, "상의", category, color)
            os.makedirs(dst_folder, exist_ok=True)

            shutil.copy(src_path, os.path.join(dst_folder, filename))

    # ▶ 하의 copy
    if not pd.isna(row["하의_카테고리"]):
        filename = f"{style}_{ident}_하의.jpg"
        src_path = os.path.join(cropped_dir, "하의", filename)

        if os.path.exists(src_path):
            category = safe_str(row["하의_카테고리"])
            color = safe_str(row["하의_색상"])

            dst_folder = os.path.join(output_dir, "하의", category, color)
            os.makedirs(dst_folder, exist_ok=True)

            shutil.copy(src_path, os.path.join(dst_folder, filename))

print("=== 원본 보존 + 사본 분류 완료! ===")


=== 원본 보존 + 사본 분류 완료! ===


### 3. [분석용] 상,하의/원피스 table split

In [51]:
# 1. 원피스 데이터
dress_exist_df = df_filtered_2[df_filtered_2["렉트좌표_원피스"].notna()]

# 컬럼명 공백 제거 (안전장치)
dress_exist_df.columns = dress_exist_df.columns.str.strip()

# 반드시 포함할 기본 컬럼
base_cols = ["식별자", "파일명", "스타일", "서브스타일"]

# 원피스 관련 컬럼
onepiece_cols = [
    col
    for col in dress_exist_df.columns
    if col == "렉트좌표_원피스" or col.startswith("원피스_")
]

# 최종 컬럼 리스트 (중복 제거)
selected_cols = [
    col for col in base_cols if col in dress_exist_df.columns
] + onepiece_cols

# 추출
onepiece_df = dress_exist_df[selected_cols]

print("Shape:", onepiece_df.shape)

Shape: (1405, 14)


In [52]:
# 2. 상의, 하의 데이터
dress_nan_df = df_filtered_2[df_filtered_2["렉트좌표_원피스"].isna()]

# 컬럼명 공백 제거 (안전장치)
dress_nan_df.columns = dress_nan_df.columns.str.strip()

# 반드시 포함할 기본 컬럼
base_cols = ["식별자", "파일명", "스타일", "서브스타일"]

# 상의 관련 컬럼
top_cols = [
    col
    for col in dress_nan_df.columns
    if col == "렉트좌표_상의" or col.startswith("상의_")
]

# 하의 관련 컬럼
bottoms_cols = [
    col
    for col in dress_nan_df.columns
    if col == "렉트좌표_하의" or col.startswith("하의_")
]

# 최종 컬럼 리스트 (중복 제거)
selected_cols = (
    [col for col in base_cols if col in dress_nan_df.columns] + top_cols + bottoms_cols
)

# 추출
top_bottom_df = dress_nan_df[selected_cols]

print("Shape:", top_bottom_df.shape)

Shape: (4704, 22)


In [53]:
onepiece_df.to_csv("C:/Users/Admin/Desktop/PROJ/data/02_cleaned/clothes/0.onepiece_only.csv",
    index=False,
    encoding="utf-8-sig"
)

top_bottom_df.to_csv("C:/Users/Admin/Desktop/PROJ/data/02_cleaned/clothes/0.top_bottom_only.csv.csv",
    index=False,
    encoding="utf-8-sig"
)